В даному ноутбуку вказано учбову реалізацію запропонованої в випускній-кваліфікаційній роботі
україномовної моделі для запрононованої та реалізованої в випускній-кваліфікаційній роботі системи аналізу настроїв у текстових повідомленнях.

Був використаний датасет з нечіткою класифікацією типу позитивний/негативний.

Сама система знаходиться в окремому ноутбуці за наступним посиланням
https://colab.research.google.com/drive/1ZmlgVkDFcaf7xYx6UNqIIsH7YCz0MVJs?usp=sharing

# Train your ukranian emotion analysis model here.

STEP #1: Run next cell to setup needed libraries for work.

In [ ]:
# Cell 1: Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense, LSTM
from tensorflow.keras.models import Sequential
from google.colab import drive

In [ ]:
# Cell 2: Mount Google Drive to access files
drive.mount('/content/drive')

STEP #2: Load your data: train, validate, test files.

In [ ]:
# Cell 3: Load your data
def load_file(file_path):
    """
    Function to load sentiment data from a CSV file.

    Args:
    file_path (str): Path to the CSV file.

    Returns:
    list, list: List of texts and corresponding labels.
    """
    df = pd.read_csv(file_path)
    texts = df['word'].tolist()
    labels = df['pos_neg'].tolist()
    return texts, labels

train_texts, train_labels = load_file('/content/drive/MyDrive/sentiment_ukr.csv')


NameError: name 'load_file' is not defined

In [ ]:
#(Op) Uncomment & load your file manually:
#from google.colab import files
#uploaded = files.upload()

STEP #3: Use next cell to prepare data for proper usage for build-up model

In [ ]:
# Cell 4: Split data into train, validation, and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(train_texts, train_labels, test_size=0.2, random_state=42)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.2, random_state=42)

In [ ]:
# Convert labels to numerical values
label_to_index = {'sadness': 0, 'joy': 1, 'anger': 2, 'fear': 3}
train_labels = [label_to_index[label] for label in train_labels]
val_labels = [label_to_index[label] for label in val_labels]
test_labels = [label_to_index[label] for label in test_labels]

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_texts)
train_sequences = tokenizer.texts_to_sequences(train_texts)
val_sequences = tokenizer.texts_to_sequences(val_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

# Pad the sequences
max_length = max([len(seq) for seq in train_sequences + val_sequences + test_sequences])
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post')
val_padded = pad_sequences(val_sequences, maxlen=max_length, padding='post')
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post')

STEP #4: Training your model: may take some time

In [ ]:
# Cell 4: Build and train the RNN (LSTM) model
model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, 128, input_length=max_length))
model.add(LSTM(64))
model.add(Dense(64, activation='relu'))
model.add(Dense(4, activation='softmax'))  # 4 classes: sadness, joy, anger, fear

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_padded, np.array(train_labels), epochs=10, batch_size=32, validation_data=(val_padded, np.array(val_labels)))

In [ ]:
test_loss, test_acc = model.evaluate(test_padded, np.array(test_labels))
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}')

STEP #5: Test your model on real examples:

In [ ]:
# Cell 5: Test the model with user input
user_input = input("Enter a text to test the model: ")
user_sequence = tokenizer.texts_to_sequences([user_input])
user_padded = pad_sequences(user_sequence, maxlen=max_length, padding='post')
prediction = model.predict(user_padded)
predicted_label = np.argmax(prediction)
label_index = {0: 'sadness', 1: 'joy', 2: 'anger', 3: 'fear'}
print(f"Predicted emotion: {label_index[predicted_label]}")

STEP #6: Save your model and download it to use on colab/local system in .pb format.

In [ ]:
import pickle
with open('tokenizer.pkl', 'wb') as tokenizer_file:
    pickle.dump(tokenizer, tokenizer_file)

model.save('nlp.h5')